[View in Colaboratory](https://colab.research.google.com/github/JonathanBheri/Machine-learning/blob/master/IMdB_using_keras_from_TensorBoard.ipynb)

# Setting up Public Host for TensorBoard

In [1]:
#@title Installing Ngrok to Pipeline TensorBoard to public host { display-mode: "form" }
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip

--2018-10-25 22:49:11--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.235.97.255, 34.238.3.58, 52.20.145.121, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.235.97.255|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5363700 (5.1M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]   5.11M  9.69MB/s    in 0.5s    

2018-10-25 22:49:11 (9.69 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [5363700/5363700]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [3]:
#@title Pipelining TensorBoard output from Local Host to Public Host (Link Below) files at path/gate { vertical-output: true }
from os.path import abspath
LOG_DIR = abspath('gate')
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://174.ngrok.io


# Keras for Imdb dataset

In [4]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Convolution1D, Flatten, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import TensorBoard


# Using keras to load the dataset with the top_words
top_words = 10000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)


# Pad the sequence to the same length
max_review_length = 1600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)


# Using embedding from Keras
embedding_vecor_length = 300
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))


# Convolutional model (3x conv, flatten, 2x dense)
model.add(Convolution1D(64, 3, padding='same'))
model.add(Convolution1D(32, 3, padding='same'))
model.add(Convolution1D(16, 3, padding='same'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(180,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))


# Log to tensorboard
tensorBoardCallback = TensorBoard(log_dir='./gate', write_graph=True)
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=3, callbacks=[tensorBoardCallback], batch_size=64)


Using TensorFlow backend.


17465344/17464789 [==============================] - 1s 0us/step
Epoch 1/3
25000/25000 [==============================] - 27s 1ms/step - loss: 0.3827 - acc: 0.8204
Epoch 2/3
25000/25000 [==============================] - 24s 972us/step - loss: 0.2102 - acc: 0.9169
Epoch 3/3
25000/25000 [==============================] - 24s 976us/step - loss: 0.1188 - acc: 0.9565


In [5]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 85.54%
